In [2]:
import GEOparse
import pandas as pd
import numpy as np

In [3]:
# get softfile from geo and store in dir
gse = GEOparse.get_GEO(geo="GSE196793", destdir="./")

02-Dec-2022 10:40:20 DEBUG utils - Directory ./ already exists. Skipping.
02-Dec-2022 10:40:20 INFO GEOparse - Downloading ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE196nnn/GSE196793/soft/GSE196793_family.soft.gz to ./GSE196793_family.soft.gz
100%|██████████| 5.75k/5.75k [00:00<00:00, 9.38kB/s]
02-Dec-2022 10:40:22 DEBUG downloader - Size validation passed
02-Dec-2022 10:40:22 DEBUG downloader - Moving /var/folders/0r/_6trl51948x_vd90czsk02t40000gq/T/tmpv2j78yt4 to /Users/jdickinson/Documents/SomaData/tutorial/rnaseq-analysis/GSE196793_family.soft.gz
02-Dec-2022 10:40:22 DEBUG downloader - Successfully downloaded ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE196nnn/GSE196793/soft/GSE196793_family.soft.gz
02-Dec-2022 10:40:22 INFO GEOparse - Parsing ./GSE196793_family.soft.gz: 
02-Dec-2022 10:40:22 DEBUG GEOparse - DATABASE: GeoMiame
02-Dec-2022 10:40:22 DEBUG GEOparse - SERIES: GSE196793
02-Dec-2022 10:40:22 DEBUG GEOparse - PLATFORM: GPL18573
02-Dec-2022 10:40:22 DEBUG GEOparse - SAMPLE: GSM

In [4]:
# Write metadata to a dictionary to later save to a pandas df
# create two empty dictionaries to store names & characteristics
sample_names = []
characteristics = []

# Unpack data from soft file
print("Saving metadata:")
for gsm_name, gsm in gse.gsms.items():
    for key, value in gsm.metadata.items():
        if key.startswith("title"):
            sample_names.append(value)
        if key.startswith('characteristics'):
            characteristics.append(value)

Saving metadata:


In [5]:
# check to make sure unpacking was successful
print(sample_names[0:5])
print(characteristics[0:4])

[['1125-0'], ['4115-0'], ['4053-0'], ['1135-0'], ['4025-0']]
[['sample timing: pre-vaccination', 'frailty status: non-frail', 'age: 68.5'], ['sample timing: pre-vaccination', 'frailty status: frail', 'age: 88.1'], ['sample timing: pre-vaccination', 'frailty status: frail', 'age: 70.6'], ['sample timing: pre-vaccination', 'frailty status: non-frail', 'age: 66.6']]


In [6]:
# Make sample names strings and not a list of a string
for i in range(len(sample_names)):
    sample_names[i] = ''.join(sample_names[i])

# check
print(sample_names)

['1125-0', '4115-0', '4053-0', '1135-0', '4025-0', '0029-0', '0148-0', '0226-0', '0225-0', '4007-0', '4162-0', '4153-0', '0258-0', '1130-0', '1116-0', '4050-0', '4149-0', '0104-0', '0147-0', '1235-0', '4108-0', '3414-0', '4026-0', '4110-0', '4163-0', '0107-0', '0316-0', '4113-0', '1125-3', '4115-3', '4053-3', '1135-3', '4025-3', '0029-3', '0148-3', '0226-3', '0225-3', '4007-3', '4162-3', '4153-3', '0258-3', '1130-3', '1116-3', '4050-3', '4149-3', '0104-3', '0147-3', '1235-3', '4108-3', '3414-3', '4026-3', '4110-3', '4163-3', '0107-3', '0316-3', '4113-3', '1125-7', '4115-7', '4053-7', '1135-7', '4025-7', '0029-7', '0148-7', '0226-7', '0225-7', '4007-7', '4162-7', '4153-7', '0258-7', '1130-7', '1116-7', '4050-7', '4149-7', '0104-7', '0147-7', '1235-7', '4108-7', '3414-7', '4026-7', '4110-7', '4163-7', '0107-7', '0316-7', '4113-7']


In [7]:
# create separate lists for sample timing, frailty, age, from characteristics
sample_timing = []
frailty_status = []
age = []
for entry in characteristics:
    for sub in entry:
        if len(sub) >= 30:
            sample_timing.append(sub[15::])
            continue
        if len(sub) >= 21:
            frailty_status.append(sub[16::])
            continue
        if len(sub) <= 9:
            age.append(sub[5::])


In [8]:
# Create pandas df from disparate lists
# first create a dict from lists
d = {
    'SampleID': sample_names, 
    'SampleTiming': sample_timing, 
    'FrailtyStatus': frailty_status,
    'Age': age
    }

metadata = pd.DataFrame(d)
metadata.head()

,SampleID,SampleTiming,FrailtyStatus,Age
0,1125-0,pre-vaccination,non-frail,68.5
1,4115-0,pre-vaccination,frail,88.1
2,4053-0,pre-vaccination,frail,70.6
3,1135-0,pre-vaccination,non-frail,66.6
4,4025-0,pre-vaccination,frail,82.9


In [10]:
metadata.to_csv("metadataCleaned.csv", index=False)

In [9]:
# load in counts data
counts = pd.read_csv('raw_data/GSE196793_htseq_counts.txt', sep="\t", index_col="ensembl_gene_id")


In [10]:
counts.head(3)

,0029-0,0029-3,0029-7,0104-0,0104-3,0104-7,0107-0,0107-3,0107-7,0147-0,...,4149-7,4153-0,4153-3,4153-7,4162-0,4162-3,4162-7,4163-0,4163-3,4163-7
ensembl_gene_id,,,,,,,,,,,,,,,,,,,,,
ENSG00000000005,0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000000419,228,161,149,324,307,743,267,191,260,168,...,316,612,190,176,314,191,262,300,306,328
ENSG00000000457,1041,813,764,1339,1205,1602,920,826,732,1066,...,979,1662,316,902,1069,872,776,1191,1165,1206


In [11]:
# Remove unnecessary columns
counts = counts.T
counts.drop(
    columns=['__no_feature', '__ambiguous', '__too_low_aQual', '__not_aligned', '__alignment_not_unique'],
    inplace=True
    )

counts = counts.T

In [18]:
# Need to normalize the dataset to counts per million (CPM)
# Per sample equation = counts / library counts sum * 1x10^6

# define function to perform normalization
def normalize_counts(df):

    # drop rows with less than 10 counts
    df = df[df.sum(axis=1)>=10]

    # get column names
    names = list(df.columns)

    for name in names:
        total = sum(df[name])
        df[name] = df[name].apply(lambda x : (x / total) * 1000000)

    df = df.round(decimals=0)

    return df

counts = normalize_counts(counts)
counts.head(3)


/var/folders/0r/_6trl51948x_vd90czsk02t40000gq/T/ipykernel_25133/3429800446.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[name] = df[name].apply(lambda x : (x / total) * 1000000)


,0029-0,0029-3,0029-7,0104-0,0104-3,0104-7,0107-0,0107-3,0107-7,0147-0,...,4149-7,4153-0,4153-3,4153-7,4162-0,4162-3,4162-7,4163-0,4163-3,4163-7
ensembl_gene_id,,,,,,,,,,,,,,,,,,,,,
ENSG00000000419,7.0,7.0,5.0,11.0,9.0,20.0,7.0,4.0,8.0,6.0,...,12.0,19.0,29.0,6.0,10.0,8.0,10.0,9.0,9.0,10.0
ENSG00000000457,31.0,37.0,28.0,47.0,36.0,43.0,25.0,18.0,22.0,36.0,...,38.0,52.0,48.0,32.0,32.0,39.0,30.0,34.0,33.0,36.0
ENSG00000000460,7.0,10.0,9.0,10.0,10.0,10.0,8.0,6.0,6.0,14.0,...,12.0,11.0,15.0,10.0,9.0,10.0,11.0,14.0,12.0,14.0


In [19]:
counts.to_csv('normalized_counts.csv')

In [20]:
# Need to merge metadata and normalized and raw counts for a merged file
counts_transposed = counts.T
counts_transposed.head()

ensembl_gene_id,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,ENSG00000001460,ENSG00000001461,...,ENSG00000284688,ENSG00000284690,ENSG00000284691,ENSG00000284697,ENSG00000284707,ENSG00000284713,ENSG00000284719,ENSG00000284734,ENSG00000284738,ENSG00000284747
0029-0,7.0,31.0,7.0,385.0,3.0,12.0,30.0,39.0,3.0,52.0,...,1.0,8.0,8.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0
0029-3,7.0,37.0,10.0,401.0,3.0,15.0,34.0,46.0,2.0,47.0,...,0.0,7.0,6.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0
0029-7,5.0,28.0,9.0,486.0,3.0,13.0,31.0,39.0,2.0,54.0,...,0.0,10.0,11.0,0.0,2.0,1.0,0.0,0.0,0.0,2.0
0104-0,11.0,47.0,10.0,320.0,4.0,14.0,30.0,63.0,3.0,91.0,...,0.0,2.0,10.0,0.0,2.0,1.0,1.0,1.0,1.0,1.0
0104-3,9.0,36.0,10.0,203.0,3.0,13.0,47.0,56.0,3.0,80.0,...,0.0,2.0,9.0,1.0,2.0,1.0,1.0,1.0,0.0,0.0


In [21]:
# Reset the index and rename the ensemble gene id column to sampleid
counts_transposed.reset_index(inplace=True)
counts_transposed.rename_axis(None, axis=1, inplace=True)
counts_transposed.rename(columns={'index': 'SampleID'}, inplace=True)

In [22]:
merged_df = counts_transposed.merge(metadata, on='SampleID')

In [23]:
merged_df.head()

,SampleID,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,ENSG00000001460,...,ENSG00000284697,ENSG00000284707,ENSG00000284713,ENSG00000284719,ENSG00000284734,ENSG00000284738,ENSG00000284747,SampleTiming,FrailtyStatus,Age
0,0029-0,7.0,31.0,7.0,385.0,3.0,12.0,30.0,39.0,3.0,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,pre-vaccination,frail,73.6
1,0029-3,7.0,37.0,10.0,401.0,3.0,15.0,34.0,46.0,2.0,...,0.0,1.0,1.0,0.0,1.0,0.0,1.0,3 days post-vaccination,frail,73.6
2,0029-7,5.0,28.0,9.0,486.0,3.0,13.0,31.0,39.0,2.0,...,0.0,2.0,1.0,0.0,0.0,0.0,2.0,7 days post-vaccination,frail,73.6
3,0104-0,11.0,47.0,10.0,320.0,4.0,14.0,30.0,63.0,3.0,...,0.0,2.0,1.0,1.0,1.0,1.0,1.0,pre-vaccination,non-frail,62.2
4,0104-3,9.0,36.0,10.0,203.0,3.0,13.0,47.0,56.0,3.0,...,1.0,2.0,1.0,1.0,1.0,0.0,0.0,3 days post-vaccination,non-frail,62.2


In [47]:
merged_df.to_csv('NormalizedCountsMetadata.csv')